# Prepocessing

In [73]:
import re
import string
import operator
from rusenttokenize import ru_sent_tokenize
from collections import Counter
from razdel import tokenize as razdel_tokenize
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

In [2]:
class Preprocessor():
    """
        a class to preprocess a txt file availabe through the path
    """
    def __init__(self, path):
        self.path = path # a path to a txt file
        self.content = ""
        self.clean = ""
        
    def getFileContent(self):
        """
            loads the content of the file
        """
        with open(self.path, 'r', encoding="utf8") as f: # open in readonly mode
            # do your stuff
            self.content = f.read()
            
    def removeJunk(self):
        """
            removes tags and other garbage with re module
        """
        pattern = re.compile(
                (r'(<[^>]+>)' # tags
                 r'|'
                 r'([A-Za-z0-9\+/]\.?\-?)' # latin chars, digits and irrelevant punkt 
                 ), re.VERBOSE)
        tmp = re.sub(pattern, ' ', self.content)
        tmp = re.sub('(  +)', ' ', tmp) # remove space sequences
        tmp = re.sub('(\s\s+)', '. ', tmp) # remove blank lines.
    
        self.clean = tmp
        
    def getSentences(self, text) -> list:
        """
            uses DeepPavlov's rusentokenize to split the text into sentences
            by default it uses a self.clean attribute
            received after self.removeJunk()
        """
        return(ru_sent_tokenize(text))
    
    
    def tokenize(self, txt) -> list:
        """
            removes any punctuation from a string
            return a list of tokenized lowercase string
        """
        punkt = string.punctuation + '«»—…–“”'
        tokens = []
        
        for word in list(razdel_tokenize(txt)):
            token = word.text.strip(punkt).lower() # remove punctuation
            if token == "": # skip empty elements
                continue
            tokens.append(token)

        return(tokens)
    
    def filter_tokens_by_length(self, tokens, relation, length):
        """
            uses str(operators) and length of the string to filter list of strings 
        """
        ops = {'>': operator.gt,
               '<': operator.lt,
               '>=': operator.ge,
               '<=': operator.le,
               '=': operator.eq}
        
        return([token for token in tokens if ops[relation](len(token), length)])
    
    
    def group_sentences(self, tokens_by_sentences) -> list:
        """
            receives lists of tokens grouped by sentences
            turns them into lowercase string with no punctuation
        """
        sentences = []
        for sentence in tokens_by_sentences:
            sentences.append(" ".join(sentence)) # turns tokens into a string
        
        return(sentences)
    
    def find_common(self, elems, top=None) -> list:
        """
            receives list of elements
            returns a list of counters
            [(elem, 7)]
        """
        return(Counter(elems).most_common(top))
    
    def getStems(self, tokens) -> list:
        """
            get [(word: stem)] using SnowballStemmer by nltk
        """
        stemmer = SnowballStemmer('russian')
        return([(word, stemmer.stem(word)) for word in tokens])

### Задание 1
Отчистите текст от мусора (тэгов, хешей и тп) с помощью регулярных выражений. Постарайтесь убрать весь мусор, но если что-то удалить не получается, то не мучайтесь. Главное, чтобы мусор не проявлялся в результатах следующих заданий.

In [3]:
preproc = Preprocessor('zhivago.txt')
preproc.getFileContent()
preproc.removeJunk()
print(preproc.clean[0:500])
print('\n')
print(preproc.clean[-500:-1])

 Борис Леонидович Пастернак. Доктор Живаго. «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни.. «Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. Человек, который так пишет, бесконечн


тынь. Сейчас должно написанное сбыться, Пускай же сбудется оно. Аминь. Ты видишь, ход веков подобен притче И может загореться на ходу. Во имя страшного ее величья Я в добровольных муках в гроб сойду. Я в гроб сойду и в третий день восстану, И, как сплавляют по реке плоты, Ко мне на суд, как баржи каравана, Столетья поплывут из темноты».. Примечания. В восторге. (Здесь и далее с французского.). Большой круг! Китайская цепочка!. Вальс, пожалуйста!. На три счета, на два счета.. Шиворот-навыворот

### Задание 2
Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize. 
Ответьте на следующие вопросы:

1) есть ли в тексте повторяющиеся корректные предложения? если да то какие? (если находится мусор то вернитесь к заданию 1 и постарайтесь избавиться от него)

2) какой самый частотный токен в тексте длиннее 6 символов?

In [50]:
tokens = preproc.tokenize(preproc.clean)
sentences = preproc.getSentences(preproc.clean)
tokens_by_sentence = [preproc.tokenize(sentence) for sentence in sentences]

In [102]:
print(sentences[0:4])
print(tokens_by_sentence[0:2])
print(tokens[0:5])

['Борис Леонидович Пастернак.', 'Доктор Живаго.', '«Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни.', 'Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни..']
[['борис', 'леонидович', 'пастернак'], ['доктор', 'живаго']]
['борис', 'леонидович', 'пастернак', 'доктор', 'живаго']


In [103]:
# most common token longer than 6
common_token = preproc.find_common(
    preproc.filter_tokens_by_length(tokens, '>', 6),
    1)
# common sentences
common_sentence = preproc.find_common(
    preproc.group_sentences(
        preproc.filter_tokens_by_length(tokens_by_sentence, '>', 6)
    ), 1)

In [104]:
print(common_sentence)
print(common_token)

[('единственно живое и яркое в вас это то что вы жили в одно время со мной и меня знали', 2)]
[('андреевич', 289)]


### Задание 3

Сделайте стемминг и найдите по несколько частотных примеров на каждый из видов ошибок:
1) два разных слова ошибочно свелись к одинаковой основе
('прем', ['премию', 'премило'], ('лебед', ['лебедью', 'лебедь', 'лебеды'])

2) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов) 
('дружок', 'дружок'), ('грешат', 'грешат')

***не делайте это задание полностью вручную, напишите правила, которые отловят потенциальные ошибки и выберите из них***

In [4]:
stems = preproc.getStems(preproc.tokenize(preproc.clean))

In [29]:
false_relatives = {} # {stem: [tokens]}

for stem in stems:
    if stem[1] in false_relatives.keys():
        false_relatives[stem[1]].append(stem[0])
        continue
    false_relatives[stem[1]] = stem[0].split()

filtered = []

for stem, forms in false_relatives.items():
    if len(forms) > 1:
        if len(set(forms)) == len(forms):
            filtered.append((stem, forms))

print(filtered[0])

('прем', ['премию', 'премило'])


In [43]:
no_change = [stem for stem in stems if stem[0] == stem[1] and len(stem[0]) > 4]
print(no_change[525])

('дружок', 'дружок')


### Задание 4
Проанализируйте список стоп-слов из нлтк (для русского). Какие ещё слова вы бы туда добавили? (5 слов будет достаточно, не забудьте аргументировать ваш выбор)

In [72]:
from nltk.corpus import stopwords
stops = stopwords.words('russian')

"""
# опираясь на текст пастернака можно добавить 
# некоторые из 200 наиболее употребимых токенов из произведения
# будет зависеть от анализируемого текста и задач
"""
to_add = ["пока", # 93
          "который", # 104
          "что-то", # 123
          "нам", # 66
          "несколько", # 115
         ]

### Задание 5

Предобработайте текст двумя способами:
1) лемматизируйте токены с помощью pymorphy2
2) лемматизируйте текст с помощью mystem3 

Ответьте на вопрос:
Что в данном случае лучше для лемматизации mystem или pymorphy?

Важно, чтобы ответы на вопросы были аргументированы (как минимум три аргумента).  Анализируйте результаты с языковой точки зрения. Скорость работы простота интерфейса не являются преимуществами или недостатками в рамках этого задания. Идеальный аргумент - библиотека x неправильно обрабатывает вот такое слово/класс слов.

### Ответ:

MyStem лучше справляется с существительными
1) слово "суть" остается существительным, тогда как PyMorphy переводит его в инфинитив "быть"
2) слово "денег" переходит в "деньги", тогда как PyMorphy переводит его в "деньга"
3) слово "неволю" переходит в "неволя", тогда как PyMorphy переводит его в инфинитив "неволить"
4) слово "иней" переходит в "иней", тогда как PyMorphy переводит его в "иня"

MyStem приводит глаголы к несовершенному виду, тогда как PyMorphy к совершенному (скорее замечание, а не проблема)

In [166]:
class Morpher():
    
    def __init__(self, morph_type):
        if morph_type == 'pymorphy2':
            self.morpho = MorphAnalyzer() 
            self.cash = {}
            self.analyze = self.usePymorphy

        elif morph_type == 'mystem':
            self.mystem = Mystem()
            self.analyze = self.useMystem
        else:
            return(ValueError)
        
        self.mode = morph_type
            
    def useMystem(self, words) -> list:
        """ 
            receives list of raw words
        """
        text = ' '.join(words)
        tokens = self.mystem.analyze(text)
        res = {}
        for t in tokens:
            if 'analysis' in t.keys():
                if t['analysis'] != []:
                    res[t['text']] = t['analysis'][0]['lex']
                else:
                    res[t['text']] = t['text']
        return(res)    

    def usePymorphy(self, words) -> list:
        """ 
            receives list of raw words
        """
        res={}
        for w in words:
            if w in self.cash:
                res[w] = self.cash[w]
            else:
                r=self.morpho.parse(w)[0].normal_form
                res[w] = r
                self.cash[w]=r
        return(res)

In [167]:
pymorpher = Morpher('pymorphy2')
mymorpher = Morpher('mystem')

In [179]:
tokens_by_pymorphy = pymorpher.analyze(tokens[0:10000])
tokens_by_mystem = mymorpher.analyze(tokens[0:10000])

In [171]:
print(tokens_by_pymorphy)
print(tokens_by_mystem)

{'борис': 'борис', 'леонидович': 'леонидович', 'пастернак': 'пастернак', 'доктор': 'доктор', 'живаго': 'живаго', 'итоговое': 'итоговый', 'произведение': 'произведение', 'бориса': 'борис'}
{'борис': 'борис', 'леонидович': 'леонидович', 'пастернак': 'пастернак', 'доктор': 'доктор', 'живаго': 'живаго', 'итоговое': 'итоговый', 'произведение': 'произведение', 'бориса': 'борис'}


In [188]:
difference = {}
for k, v in tokens_by_pymorphy.items():
    try:
        if tokens_by_mystem[k] != v:
            difference[k + "_" + 'PM'] = v
            difference[k + "_" + 'MS'] = tokens_by_mystem[k]
    except Exception:
        continue